In [ ]:
# %pip install bingmaps

In [1]:
import bingmaps
from bingmaps.apiservices import LocationByQuery
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_csv('8_country.csv')
pd.options.display.max_columns = 100
df

/var/folders/1p/747b_4cn70v5h3223zdf0k180000gp/T/ipykernel_33299/3411018608.py:1: DtypeWarning: Columns (55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('8_country.csv')


,make_model,short_description,make,model,location,price,body_type,type,doors,country_version,offer_number,warranty,mileage,first_registration,gearbox,fuel_type,colour,paint,desc,seller,seats,power,engine_size,gears,co2_emissions,manufacturer_colour,drivetrain,cylinders,fuel_consumption,comfort_convenience,entertainment_media,safety_security,extras,empty_weight,model_code,general_inspection,last_service,full_service_history,non_smoker_vehicle,emission_class,emissions_sticker,upholstery_colour,upholstery,production_date,previous_owner,other_fuel_types,power_consumption,energy_efficiency_class,co2_efficiency,fuel_consumption_(wltp),co2_emissions_(wltp),available_from,taxi_or_rental_car,availability,last_timing_belt_change,electric_range_(wltp),power_consumption_(wltp),battery_ownership,desc_lang,desc_lang2,desc_en,address,latitude,longitude,fuel_consumption_comb,fuel_consumption_city,fuel_consumption_country,country
0,Mercedes-Benz A 160,CDi,Mercedes-Benz,A 160,"P.I. EL PALMAR C/FORJA 6, 11500 PUERTO DE SAN...",16950,Compact,Used,5.0,"['\n, Spain, \n']","['\n, 7468157, \n']",24,120200.0,2016-06-01,Manual,Diesel,Black,Metallic,"['Precio al contado: 16950 euros', 'Vehículo c...",Dealer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 16950 euros',' vehicle with main...","Calle Forja 6, 11500 El Puerto de Santa María,...",36.618987,-6.205012,NaN,NaN,NaN,Spain
1,Mercedes-Benz EQE 350,350+,Mercedes-Benz,EQE 350,"APARTADO DE CORREOS 1032, 26140 LOGROÑO, ES",80900,Compact,Pre-registered,4.0,"['\n, Spain, \n']","['\n, 7054863, \n']",24,5000.0,2022-06-01,Automatic,Electric,NaN,NaN,"['Precio al contado: 88900 euros', 'AUTO OJA S...",Dealer,5.0,215 kW (292 hp),1.0,9.0,0.0,Negro Obsidiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 88900 euros',' Auto Oja S.A. - O...","Calle Logroño, 28649 Rozas de Puerto Real, Mad...",40.315198,-4.474838,NaN,NaN,NaN,Spain
2,Mercedes-Benz A 45 AMG,S 4Matic+ 8G-DCT,Mercedes-Benz,A 45 AMG,"PORT. TARRACO, MOLL DE LLEVANT, Nº 5, LOC. 6-8...",69900,Compact,Used,5.0,"['\n, Spain, \n']","['\n, 7410454, \n']",12,18900.0,2020-07-01,Automatic,Gasoline,Grey,NaN,"['Gris MANUFAKTUR mountaingrau magno', 'Linea/...",Dealer,5.0,310 kW (421 hp),1991.0,8.0,NaN,Mountaingrau magno,4WD,4.0,"[['8.4 l/100 km (comb.)'], ['10.9 l/100 km (ci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Gray Manufakur Mountingrau Magno', 'Line/Sty...","Muelle de Llevant, 43004 Tarragona, Spain",41.106726,1.249235,8.4,10.9,5.8,Spain
3,Mercedes-Benz A 35 AMG,4Matic+ 7G-DCT,Mercedes-Benz,A 35 AMG,"Carrer de Provença, 31 Local, 8029 BARCELONA, ES",46990,Compact,Used,5.0,"['\n, Spain, \n']","['\n, 7464503, \n']",12,18995.0,2020-01-01,Automatic,Gasoline,Yellow,NaN,"['Precio al contado: 48990 euros', 'BONIFICACI...",Dealer,5.0,225 kW (306 hp),1991.0,7.0,NaN,Amarillo Sol,4WD,4.0,"[['7.3 l/100 km (comb.)'], ['9.3 l/100 km (cit...","['Air conditioning, Automatic climate control,...",['USB'],"['ABS, Central door lock, Driver-side airbag, ...",['Sport seats'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 48990 euros',' Bonus to finance:...","Calle Provenza, Valle de los Molinos, 45200 Gu...",20.848540,-103.428946,7.3,9.3,6.1,Mexico
4,Mercedes-Benz A 45 AMG,200CDI BE Line 4M 7G-DCT,Mercedes-Benz,A 45 AMG,"CARRIL ARAGONES 4, 30007 CASILLAS, ES",16800,Compact,Used,5.0,"['\n, Spain, \n']","['\n, 7127931, \n']",12,197000.0,2015-09-01,Automatic,Diesel,White,Metallic,"['Precio al contado: 17400 euros', 'Vehículo e...",Dealer,5.0,100 kW (136 hp),2143.0,7.0,NaN,NaN,4WD,4.0,"[['4.9 l/100 km (comb.)'], ['5.7 l/100 km (cit...","['Air conditioning, Electrical side mirrors, M...","['Bluetooth, CD

In [8]:
# key -> https://www.bingmapsportal.com/Application
bingmaps_key = ''
def getCountry(x):
    try:
        address = x

        # set up the query parameters
        params = {
            'q': address,
            'key': bingmaps_key,
        }

        # use the geocode API to get the corrected address
        response = LocationByQuery(params)

        # print the corrected address
        if response.status_code == 200:
            data = json.loads(response.response)
            try:
                country_region = data['resourceSets'][0]['resources'][0]['address']['countryRegion']
                return country_region
            except:
                return np.nan
        else:
            return np.nan
    except:
        return np.nan

In [9]:
getCountry(df.iloc[6301,:].address)

'Netherlands'

In [ ]:
df['country'] = df.location.parallel_apply(lambda x: getCountry(x))

In [84]:
df[['location','country']].drop_duplicates().dropna().reset_index(drop=True).to_csv('country.csv', index = False)

In [21]:
df2 = pd.read_csv('country.csv')

In [24]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Mexico'), 'country'] = 'Spain'

In [66]:
df2.country.value_counts()

country
Germany        3009
Italy          1339
Spain          1289
Netherlands     990
Belgium         724
France          460
Austria         323
Luxembourg       26
Hungary           8
Poland            6
Bulgaria          1
Slovenia          1
Denmark           1
Estonia           1
San Marino        1
Czechia           1
Finland           1
Name: count, dtype: int64

In [65]:
df2[df2.country == 'United States']

,location,country


In [23]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Mexico'), 'country']

3       Mexico
1988    Mexico
6034    Mexico
7172    Mexico
Name: country, dtype: object

In [30]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Canada'), 'country'] = 'Spain'

In [34]:
df2.loc[(df2['location'].str.contains('FR')) & (df2['country'] == 'Reunion'), 'country'] = 'France'

In [38]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Peru'), 'country'] = 'Spain'

In [43]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Portugal'), 'country'] = 'Spain'

In [46]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'Puerto Rico'), 'country'] = 'Spain'

In [53]:
df2.loc[(df2['location'].str.contains('DE')) & (df2['country'] == 'Switzerland'), 'country'] = 'Germany'

In [54]:
df2.loc[(df2['location'].str.contains('BE')) & (df2['country'] == 'Switzerland'), 'country'] = 'Belgium'

In [61]:
df2.loc[(df2['location'].str.contains('NL')) & (df2['country'] == 'United States'), 'country'] = 'Netherlands'

In [62]:
df2.loc[(df2['location'].str.contains('DE')) & (df2['country'] == 'United States'), 'country'] = 'Germany'

In [63]:
df2.loc[(df2['location'].str.contains('IT')) & (df2['country'] == 'United States'), 'country'] = 'Italy'

In [64]:
df2.loc[(df2['location'].str.contains('ES')) & (df2['country'] == 'United States'), 'country'] = 'Spain'

In [68]:
df2[['location','country']].drop_duplicates().dropna().reset_index(drop=True).to_csv('country.csv', index = False)

In [69]:
pd.read_csv('country.csv').country.value_counts()

country
Germany        3009
Italy          1339
Spain          1289
Netherlands     990
Belgium         724
France          460
Austria         323
Luxembourg       26
Hungary           8
Poland            6
Bulgaria          1
Slovenia          1
Denmark           1
Estonia           1
San Marino        1
Czechia           1
Finland           1
Name: count, dtype: int64